In [ ]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
from pyspark import SparkFiles

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the DataFrame
df.show()

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""SELECT YEAR(DATE) as Year_Sold_4B, ROUND(AVG(price),2) AS Avg_Price FROM home_sales WHERE BEDROOMS = 4 GROUP BY 1 ORDER BY 1 ASC""").show()



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT date_built AS Home_Built_Year, ROUND(AVG(price),2) AS Avg_Price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY 1 ORDER BY 1 ASC""").show()



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""SELECT date_built AS Home_Built_Year, ROUND(AVG(price),2)AS Avg_Price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_lot >= 2000 GROUP BY 1 ORDER BY 1 ASC""").show()



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

# Get the current time
start_time = time.time()

# The code below displays the single digit 9 as part of the list
# spark.sql("SELECT view AS Home_View_Rating, ROUND(AVG(price),2) AS Avg_Price FROM home_sales WHERE price>=350000 GROUP BY 1 ORDER BY 1 DESC").show()

# This line of code runs the query and sorts correctly by left-padding the values with '0' so that number 9 is considered 09, and be sorted correctly.
spark.sql("""SELECT LPAD(view, 2, '0') AS Home_View_Rating, ROUND(AVG(price), 2) AS Avg_Price FROM home_sales WHERE price >= 350000 GROUP BY 1 ORDER BY Home_View_Rating DESC""").show()

# Print out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

# Get the current time
start_time = time.time()

# Repeating the same query as for the previous table
spark.sql("""SELECT LPAD(view, 2, '0') AS Home_View_Rating, ROUND(AVG(price), 2) AS Avg_Price FROM home_sales WHERE price >= 350000 GROUP BY 1 ORDER BY Home_View_Rating DESC""").show()

# Print out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("parket_delayed_partitioned")

In [ ]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('parket_delayed_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_delays_p')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

# Get the current time
start_time = time.time()

# Query that calculates the avg price of a home per view rating using the parquet DataFrame
spark.sql("""SELECT LPAD(view, 2, '0') AS Home_View_Rating, ROUND(AVG(price), 2) AS Avg_Price FROM p_delays_p WHERE price >= 350000 GROUP BY 1 ORDER BY Home_View_Rating DESC""").show()

# Print out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


In [ ]:
# Write out a csv file from Spark
df.write.csv('out_home_sales_revised.csv', mode='overwrite')